# Download Data

In [73]:
!pip3 install delta-sharing --quiet

In [74]:
import delta_sharing
import json
import os

In [75]:
credentials = {
	"shareCredentialsVersion": 1,
	"bearerToken": "RbGdS53IslkEvMdYD9ist9DCa9g8K9_FPtbTDPjdIx9sOMcJUkCkXUqL6Lk4AgKr",
	"endpoint": "https://nvirginia.cloud.databricks.com/api/2.0/delta-sharing/metastores/3b3b8bc9-0c8e-4df2-9caa-e04062ece4f0",
	"expirationTime": "2024-11-12T04:53:07.255Z"
}

In [76]:
file_name = "config.share"

# Write the dictionary to a JSON file in a single line
with open(file_name, 'w') as json_file:
    json.dump(credentials, json_file, separators=(',', ':'), ensure_ascii=False)

print(f"Dictionary saved as a single-line JSON in {file_name}")

Dictionary saved as a single-line JSON in config.share


In [77]:
# Point to the profile file. It can be a file on the local file system or a file on a remote storage.
profile_file = file_name

# Create a SharingClient.
client = delta_sharing.SharingClient(profile_file)

# List all shared tables.
client.list_all_tables()

[Table(name='events_1percent', share='unlock-share', schema='bronze'),
 Table(name='events', share='unlock-share', schema='bronze'),
 Table(name='events_001percent', share='unlock-share', schema='bronze'),
 Table(name='events_10percent', share='unlock-share', schema='bronze'),
 Table(name='events_01percent', share='unlock-share', schema='bronze')]

In [78]:
# Create a url to access a shared table.
# A table path is the profile file path following with `#` and the fully qualified name of a table
# (`<share-name>.<schema-name>.<table-name>`).
table_url = profile_file + "#unlock-share.bronze.events_001percent" # -> cambien el porcentaje de acuerdo a la lista de arriba

In [79]:
# Load a table as a Pandas DataFrame. This can be used to process tables that can fit in the memory.
df = delta_sharing.load_as_pandas(table_url)

In [80]:
df.head(1)

,globaleventid,sqldate,monthyear,year,fractiondate,actor1code,actor1name,actor1countrycode,actor1knowngroupcode,actor1ethniccode,...,actor2geo_featureid,actiongeo_type,actiongeo_fullname,actiongeo_countrycode,actiongeo_adm1code,actiongeo_lat,actiongeo_long,actiongeo_featureid,dateadded,sourceurl
0,1120582762,20230813,202308,2023,2023.6110,USA,UNITED STATES,USA,None,None,...,None,2,"Arkansas, United States",US,USAR,34.9513,-92.3809,AR,20230813,https://www.eldoradonews.com/news/2023/aug/13/...


# Web pages classification


In [82]:
import pandas as pd
import requests
import threading

In [83]:
urls: pd.Series = df.sourceurl  ## Url column

In [84]:
# Shared resources
connections = []
errors = []

# Lock for synchronizing access to shared resources
lock = threading.Lock()

def worker(url, index, total_urls):
    try:
        request = requests.get(url, timeout=1)
        with lock:  # Ensure thread-safe access to shared lists
            connections.append((request.status_code, url))
    except Exception as e:
        with lock:  # Ensure thread-safe access to shared lists
            errors.append(url)
    finally:
        print(f"classified [{index} of {total_urls}]", url) # Uncommend this to get log about classification

def fetch_status_codes(urls) -> tuple:
    global connections, errors
    connections = []  # Clear previous results
    errors = []

    # Create and start threads
    threads = []
    for index, url in enumerate(urls, start=1):
        thread = threading.Thread(target=worker, args=(url, index, len(urls)))
        thread.start()
        threads.append(thread)

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    return connections, errors


# Fetch status codes and errors
status_codes, error_urls = fetch_status_codes(urls)

classified [2 of 4720] https://www.azcentral.com/story/news/local/arizona-people/2023/08/13/lloyd-hopkins-hopes-to-expand-teacher-support-nonprofit-in-arizona/70404314007/
classified [4 of 4720] https://www.express.co.uk/news/royal/1801855/prince-harry-poll-royal-family-removed-line-of-succession
classified [7 of 4720] https://sundayworld.co.za/news/eskom-not-for-sale-mbalula-assures-anc/
classified [16 of 4720] https://www.indiagazette.com/news/273931510/pm-modi-spoke-less-on-manipur-during-his-speech-in-lok-sabha-sharad-pawar
classified [6 of 4720]classified [9 of 4720] https://english.almayadeen.net/news/politics/iran-announces-fms-visit-to-saudi-arabia-in-near-future
 https://www.aceshowbiz.com/news/view/00210121.html
classified [12 of 4720] https://247wallst.com/special-report/2023/08/14/the-most-mass-produced-planes-in-world-war-ii/
classified [10 of 4720] https://www.masslive.com/worcester/2023/08/worcesters-indian-lake-closing-tuesday-for-algae-treatment.html
classified [21 of 

## Classification with Time
If you want to know how many time it takes to classify urls run this

In [85]:
import threading
import requests
import time

# Shared resources
connections = []
errors = []

# Lock for synchronizing access to shared resources
lock = threading.Lock()

def worker(url, index, total_urls):
    try:
        request = requests.get(url, timeout=1)
        with lock:  # Ensure thread-safe access to shared lists
            connections.append((request.status_code, url))
    except Exception as e:
        with lock:  # Ensure thread-safe access to shared lists
            errors.append(url)
    #finally:
        #print(f"classified [{index} of {total_urls}]", url) # Uncomment this to get log about classification

def fetch_status_codes(urls) -> tuple:
    global connections, errors
    connections = []  # Clear previous results
    errors = []

    # Create and start threads
    threads = []
    for index, url in enumerate(urls, start=1):
        thread = threading.Thread(target=worker, args=(url, index, len(urls)))
        thread.start()
        threads.append(thread)

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    return connections, errors

# Measure execution time
start_time = time.time()  # Record start time

# Fetch status codes and errors
status_codes, error_urls = fetch_status_codes(urls)

end_time = time.time()  # Record end time
elapsed_time = end_time - start_time  # Calculate elapsed time

# Print results
print(f"Status Codes: {status_codes}")
print(f"Error URLs: {error_urls}")
print(f"Execution Time: {elapsed_time:.2f} seconds")


Status Codes: [(403, 'https://sundayworld.co.za/news/eskom-not-for-sale-mbalula-assures-anc/'), (200, 'https://www.azcentral.com/story/news/local/arizona-people/2023/08/13/lloyd-hopkins-hopes-to-expand-teacher-support-nonprofit-in-arizona/70404314007/'), (403, 'https://www.aceshowbiz.com/news/view/00210121.html'), (200, 'https://www.express.co.uk/news/royal/1801855/prince-harry-poll-royal-family-removed-line-of-succession'), (403, 'https://english.almayadeen.net/news/politics/iran-announces-fms-visit-to-saudi-arabia-in-near-future'), (403, 'https://247wallst.com/special-report/2023/08/14/the-most-mass-produced-planes-in-world-war-ii/'), (202, 'https://www.indiagazette.com/news/273931510/pm-modi-spoke-less-on-manipur-during-his-speech-in-lok-sabha-sharad-pawar'), (200, 'https://www.adnews.com.au/news/independent-agencies-are-investing-in-ai-and-pandc-despite-economic-headwinds'), (200, 'https://www.jta.org/2023/08/15/israel/saudi-arabia-appoints-first-palestinian-envoy-as-saudi-israel-n